# Exercise 2: Solving a cantilever beam with Finite Differences

In this exercise we will learn how to obtain the motion of a cantilever beam using Finite Differences

The equation of motion of the beam is:
$$\rho A \ddot{w} + EI w'''' = q(x,t)$$

where $w(t)$ is the deflection of the beam, $\rho$ its density, $A$ its cross-sectional area, $E$ its Young's modulus, $I$ its second moment of area and $q(x,t)$ a distributed force applied to the beam. 

The boundary conditions for a cantilever beam are:
$$ w(0) = w'(0) = \dot{w}(0) = \ddot{w}(0) = 0 $$
$$EI\ w^{'''}(L) = F_{\mathrm{ext}}(t) $$
$$EI\ w^{''}(L) = M_{\mathrm{ext}}(t) $$

## The roadmap to discretize the beam using Finite Differences

The steps needed to discretize the beam are listed next. 
1. Discretize space into $N + 1$ points.
2. Discretize the EOM of the beam. To do this, you will have to approximate the 4th order derivative with FD. Use a second order approximation. You can find the coefficients [here](https://en.wikipedia.org/wiki/Finite_difference_coefficient) (or google `finite difference coefficients' and click the Wikipedia page)}.
3. Apply the boundary conditions. Use the definition of discrete derivatives and EOM to find the relation between the ghost points and the interior points.
4. Combine all equations into matrices. 

In [1]:
pip install ipympl;

<class 'AttributeError'>: module 'pexpect' has no attribute 'TIMEOUT'

In [2]:
# Load packages
import numpy as np

### Step 1: Discretize space

In [3]:
L = 10 # length [m]
N = 100 # Number of elements
x = np.linspace(0,L,N+1)

### Step 2: Discretize the EOM of the beam

Using the coefficients for a centered FD scheme for 4th order derivatives with 2nd order accuracy we have:

$$ w_i^{''''} = \frac{w_{i-2}-4w_{i-1}+6w_i-4w_{i+1}+w_{i+2}}{\Delta x^4}$$

The equivalent scheme for the 3rd order derivative is:

$$ w_i^{'''} = \frac{-0.5w_{i-2}+w_{i-1}-w_{i+1}+0.5w_{i+2}}{\Delta x^3}$$

For the 2nd order derivative:

$$ w_i^{''} = \frac{w_{i-1}-2w_i+w_{i+1}}{\Delta x^2}$$

And for the 1st order derivative:

$$ w_i^{'} = \frac{-0.5w_{i-1}+0.5w_{i+1}}{\Delta x}$$

Replacing these expressions into the equation of motion we get the following discrete system:

$$ \rho A \ddot{w}_i + EI \frac{w_{i-2}-4w_{i-1}+6w_i-4w_{i+1}+w_{i+2}}{\Delta x^4} = q_i $$

### Step 3: Apply boundary conditions

At $x=0$ we have that $w(0)=0$, then 
$$ w_0 = 0 $$

At $x=0$ we also have that $w^{'}(0)=0$, then using the approximation of $w^{'}_i$ we get 
$$ w_{-1} = w_1 $$

Since we know the values of $w(0)$ for all times, we will not include them on the system. Using all these relations, we get the discrete equation for $i=1$ and $i=2$ as:
$$ \rho A \ddot{w}_1 + EI \frac{7w_1-4w_{2}+w_{3}}{\Delta x^4} = q_1 $$
$$ \rho A \ddot{w}_2 + EI \frac{-4w_{1}+6w_2-4w_{3}+w_{4}}{\Delta x^4} = q_2 $$

On the other side, we use the force and moments relation
$$EI\ w^{''}(L) = M_{\mathrm{ext}}(t) $$
$$EI\frac{w_{N-1}-2w_{N}+w_{N+1}}{\Delta x^2} = M_{\mathrm{ext}}(t)$$
Thus
$$w_{N+1} = \frac{\Delta x^2}{EI}M_{\mathrm{ext}}(t)-w_{N-1}+2w_{N}$$

And
$$EI\ w^{'''}(L) = F_{\mathrm{ext}}(t) $$
$$EI \frac{-0.5w_{N-2}+w_{N-1}-w_{N+1}+0.5w_{N+2}}{\Delta x^3} = F_{\mathrm{ext}}(t) $$
$$w_{N+2} = \frac{2\Delta x^3}{EI}F_{\mathrm{ext}}(t) + \frac{2\Delta x^2}{EI}M_{\mathrm{ext}}(t)+ w_{N-2} - 4w_{N-1} + 4w_N  $$

So, the discrete equations of motion for $i=N$ and $i=N-1$ are:
$$ \rho A \ddot{w}_N + EI \frac{2w_{N-2} - 4w_{N-1} + 2w_N}{\Delta x^4} = q_N - \frac{2}{\Delta x}F_{\mathrm{ext}}(t) + \frac{2}{\Delta x^2}M_{\mathrm{ext}}(t) $$

$$ \rho A \ddot{w}_{N-1} + EI \frac{w_{N-3}-4w_{N-2}+5w_{N-1}-2w_{N}}{\Delta x^4} = q_{N-1} - \frac{1}{\Delta x^2}M_{\mathrm{ext}}(t)$$



### Step 4: Matrix form

Summarizing we have the following discrete (in space) equations:

- For $i=1$:
$$ \rho A \ddot{w}_1 + EI \frac{7w_1-4w_{2}+w_{3}}{\Delta x^4} = q_1 $$
- For $i=2$:
$$ \rho A \ddot{w}_2 + EI \frac{-4w_{1}+6w_2-4w_{3}+w_{4}}{\Delta x^4} = q_2 $$
- For $i=3,...,N-2$:
$$ \rho A \ddot{w}_i + EI \frac{w_{i-2}-4w_{i-1}+6w_i-4w_{i+1}+w_{i+2}}{\Delta x^4} = q_i $$
- For $i=N-1$:
$$ \rho A \ddot{w}_{N-1} + EI \frac{w_{N-3}-4w_{N-2}+5w_{N-1}-2w_{N}}{\Delta x^4} = q_{N-1} - \frac{1}{\Delta x^2}M_{\mathrm{ext}}(t)$$
- For $i=N$:
$$ \rho A \ddot{w}_N + EI \frac{2w_{N-2} - 4w_{N-1} + 2w_N}{\Delta x^4} = q_N - \frac{2}{\Delta x}F_{\mathrm{ext}}(t) + \frac{2}{\Delta x^2}M_{\mathrm{ext}}(t) $$  

This is equivalent to the following system:
$$\rho A \left[\begin{matrix}
1&0&0&\ldots&0&0&0\\
0&1&0&\ldots&0&0&0\\
0&0&1&\ldots&0&0&0\\
\vdots&\vdots&\vdots&\ddots&\vdots&\vdots&\vdots\\
0&0&0&\ldots&1&0&0\\
0&0&0&\ldots&0&1&0\\
0&0&0&\ldots&0&0&1\\
\end{matrix}\right]\left[\begin{matrix}
\ddot{w}_1\\
\ddot{w}_2\\
\ddot{w}_3\\
\vdots\\
\ddot{w}_{N-2}\\
\ddot{w}_{N-1}\\
\ddot{w}_N\\
\end{matrix}\right]
+ \frac{EI}{\Delta x^4} \left[\begin{matrix}
7&-4&1&0&0&&0&0&0\\
-4&6&-4&1&0&\ldots&0&0&0\\
1&-4&6&-4&1&&0&0&0\\
&&\vdots&&&\ddots&&\vdots&\\
0&0&0&&1&-4&6&-4&1\\
0&0&0&\ldots&0&1&-4&5&-2\\
0&0&0& &0&0&2&-4&2\\
\end{matrix}\right]\left[\begin{matrix}
w_1\\
w_2\\
w_3\\
\vdots\\
w_{N-2}\\
w_{N-1}\\
w_N\\
\end{matrix}\right] = \left[\begin{matrix}
q_1\\
q_2\\
q_3\\
\vdots\\
q_{N-2}\\
q_{N-1} - \frac{1}{\Delta x^2}M_{\mathrm{ext}}(t)\\
q_N - \frac{2}{\Delta x}F_{\mathrm{ext}}(t) + \frac{2}{\Delta x^2}M_{\mathrm{ext}}(t) \\
\end{matrix}\right]$$

And in a compact form:
$$\boldsymbol{M}\ddot{\boldsymbol{w}}+\boldsymbol{K}\boldsymbol{w}=\boldsymbol{F}$$

In [2]:


\section{Discretized system} 
If you did everything correctly, you should have obtained to following mass matrix of size $N \times N$ (NOT $(N+1) \times (N+1)$): 
\begin{equation}
\bm{\mathrm{M}} = \rho A \bm{\mathrm{I_N}}
\end{equation}
where $\bm{\mathrm{I_N}}$ is the identity matrix of size $N \times N$. 

The stiffness matrix of size $N \times N$ should be: 
\begin{equation}
\bm{\mathrm{K}} = \frac{EI}{l^4} 
\begin{bmatrix} 
7  & -4 & 1  & 0  & 0 & 0 & 0  \\
-4 & 6  & -4 & 1  & 0 & 0 & 0  \\
1  & -4 & 6  & -4 & 1 & 0 & 0  \\
   &   &   &\cdots & &  &  \\
0  & 0 & 1 & -4 & 6  & -4 & 1  \\
0  & 0 & 0 & 1  & -4 & 5  & -2 \\
0  & 0 & 0 & 0  & 2  & -4 & 2 
\end{bmatrix}
\end{equation}
where $\begin{bmatrix} 1 &-4 &6  &-4 &1\end{bmatrix}$ repeats along the dots, with the correct amount of zero before and after. 

The forcing vector should be:
\begin{equation}
\bm{\mathrm{F}} = \begin{bmatrix} 
q_1 \\ q_2 \\ q_3 \\ \cdots \\ q_{N-2} \\ q_{N-1} + \frac{M_{\mathrm{ext}}(t)}{l^2} \\ q_N + 2 \frac{F_{\mathrm{ext}}(t)}{l} - 2 \frac{M_{\mathrm{ext}}(t)}{l^2} \\
\end{bmatrix}
\end{equation}
where $q_n = q(x_n)$. 

The derivations will be made available after the lecture. 

\section{Bonus learning} 
At this point you can program the matrices, substitute expressions for the loads and solve the system. The complexity of the model can be further increased to make it a better model of a wind turbine. You can try adding the following items and see how they change the matrices: 
\begin{itemize}
\item A point mass at the free end of the beam to capture the nacelle. 
\item The effect of axial compression $N w''$. 
\item Rather than having a clamped end, have a spring foundation to support the pipe in transverse direction. 
\end{itemize}

\end{document}

In this project you will analyse the behavior of an offshore wind turbine. 

<class 'SyntaxError'>: unexpected character after line continuation character (<ipython-input-2-e121d40da20c>, line 1)